# Setup

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Apps setting both CustomTextKey and Description in a ValidationIssue

In [ ]:
with Apps.init() as apps:
    apps.select(
        {
            "Validation issues": lambda app: app.cs.flat_map(
                lambda cs: cs.object_creations(type="ValidationIssue", fields=["CustomTextKey", "Description"])
            )
        }
    ).where(lambda app: app["Validation issues"].length > 0).table()

## IFormDataValidator.HasRelevantChanges

In [ ]:
with Apps.init() as apps:
    apps.where(lambda app: app.env == "prod").select(
        {
            "HasRelevantChanges": lambda app: app.cs.flat_map(
                lambda cs: cs.class_declarations(implements=["IFormDataValidator"])
            ).flat_map(
                lambda formDataValidator: formDataValidator.method_declarations(
                    name="HasRelevantChanges", returns="bool", modifiers=["public"]
                )
            )
        }
    ).where(lambda app: app["HasRelevantChanges"].length > 0).table()